In [1]:
import cv2
import math
import numpy as np
import time
from realsense_camera import *

# Инициализация камеры
camera = RealsenseCamera()

# Переменные для трекинга
cx, cy = 0, 0
cx_mm, cy_mm = 0, 0
prev_cx, prev_cy = 0, 0
stable_count = 0

# Параметры стабилизации
MIN_STABLE_FRAMES = 5
MAX_MOVEMENT = 15

# Параметры фильтрации контуров
MIN_CONTOUR_AREA = 500
MIN_CONTOUR_WIDTH = 20
MIN_CONTOUR_HEIGHT = 20

# Параметры камеры (должны быть получены из калибровки)
Cx = 320  # Оптический центр X
Cy = 240  # Оптический центр Y
fx = 604.602  # Фокусное расстояние X
fy = 604.162  # Фокусное расстояние Y

print("Запуск обработки видео. Нажмите 'q' для выхода.")

while True:
    # Получаем кадры
    ret, depth_frame, color_frame, depth = camera.get_frame_stream()

    if not ret:
        print("Ошибка получения кадра")
        continue

    height, width = color_frame.shape[0:2]   

    # Обработка цвета
    hsv = cv2.cvtColor(color_frame, cv2.COLOR_BGR2HSV)    

    # Цветовые диапазоны
    bin1 = cv2.inRange(hsv, (20, 100, 100), (30, 255, 255)) # желтый
    bin2 = cv2.inRange(hsv, (0, 56, 95), (71, 255, 255)) # красный
    bin3 = cv2.inRange(hsv, (35, 100, 100), (85, 255, 255)) # зеленый
    bin4 = cv2.inRange(hsv, (90, 100, 100), (130, 255, 255)) # синий
    binary = bin1 + bin2 + bin3 + bin4  

    # Морфологические операции
    kernel = np.ones((5, 5), np.uint8)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

    # Поиск контуров
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Фильтрация контуров
    valid_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < MIN_CONTOUR_AREA:
            continue

        x, y, w, h = cv2.boundingRect(contour)
        if w < MIN_CONTOUR_WIDTH or h < MIN_CONTOUR_HEIGHT:
            continue

        valid_contours.append(contour)

    # Обработка найденных контуров
    frame_display = color_frame.copy()

    if valid_contours:
        largest_contour = max(valid_contours, key=cv2.contourArea)
        moments = cv2.moments(largest_contour)

        if moments["m00"] > 0: 
            new_cx = int(moments["m10"] / moments["m00"])  
            new_cy = int(moments["m01"] / moments["m00"])

            # Проверка стабильности
            movement = math.sqrt((new_cx - prev_cx)**2 + (new_cy - prev_cy)**2)

            if movement < MAX_MOVEMENT:
                stable_count += 1
            else:
                stable_count = 0

            # Обновление координат при стабильности
            if stable_count >= MIN_STABLE_FRAMES:
                cx, cy = new_cx, new_cy

                # Преобразование в миллиметры с проверкой глубины
                if 0 <= cy < depth.shape[0] and 0 <= cx < depth.shape[1]:
                    depth_value = depth[cy, cx]
                    if depth_value > 0:
                        cx_mm = (cx - Cx) * depth_value / fx
                        cy_mm = (cy - Cy) * depth_value / fy
                    else:
                        cx_mm, cy_mm = 0, 0
                else:
                    cx_mm, cy_mm = 0, 0

            prev_cx, prev_cy = new_cx, new_cy

            # Визуализация
            cv2.drawContours(frame_display, [largest_contour], -1, (0, 255, 0), 3)
            cv2.circle(frame_display, (cx, cy), 8, (0, 0, 255), -1)
            cv2.circle(frame_display, (cx, cy), 12, (255, 255, 255), 2)

            # Отображение информации
            cv2.putText(frame_display, f'X: {cx} px', (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            cv2.putText(frame_display, f'Y: {cy} px', (10, 60), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            cv2.putText(frame_display, f'X: {cx_mm:.1f} mm', (10, 90), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            cv2.putText(frame_display, f'Y: {cy_mm:.1f} mm', (10, 120), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

            print(f"Pixel: ({cx}, {cy}), MM: ({cx_mm:.1f}, {cy_mm:.1f})")

    cv2.imshow('Color Frame', frame_display)

    # Выход по 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()

Запуск обработки видео. Нажмите 'q' для выхода.
Pixel: (0, 0), MM: (0.0, 0.0)
Pixel: (0, 0), MM: (0.0, 0.0)
Pixel: (0, 0), MM: (0.0, 0.0)
Pixel: (0, 0), MM: (0.0, 0.0)
Pixel: (0, 0), MM: (0.0, 0.0)
Pixel: (0, 0), MM: (0.0, 0.0)
Pixel: (0, 0), MM: (0.0, 0.0)
Pixel: (0, 0), MM: (0.0, 0.0)
Pixel: (522, 161), MM: (181.4, -71.0)
Pixel: (522, 161), MM: (181.4, -71.0)
Pixel: (522, 161), MM: (181.4, -71.0)
Pixel: (522, 161), MM: (181.4, -71.0)
Pixel: (522, 161), MM: (181.4, -71.0)
Pixel: (522, 161), MM: (181.4, -71.0)
Pixel: (522, 161), MM: (181.4, -71.0)
Pixel: (522, 161), MM: (181.4, -71.0)
Pixel: (522, 161), MM: (181.4, -71.0)
Pixel: (522, 161), MM: (181.4, -71.0)
Pixel: (522, 161), MM: (181.4, -71.0)
Pixel: (522, 161), MM: (181.4, -71.0)
Pixel: (404, 282), MM: (41.7, 20.9)
Pixel: (404, 282), MM: (41.5, 20.8)
Pixel: (404, 282), MM: (41.7, 20.9)
Pixel: (404, 282), MM: (41.5, 20.8)
Pixel: (404, 282), MM: (41.5, 20.8)
Pixel: (404, 282), MM: (41.5, 20.8)
Pixel: (404, 282), MM: (41.5, 20.8)
Pixe

Pixel: (404, 282), MM: (41.5, 20.8)
Pixel: (403, 282), MM: (41.0, 20.8)
Pixel: (404, 282), MM: (41.5, 20.8)
Pixel: (403, 282), MM: (41.2, 20.9)
Pixel: (404, 282), MM: (41.7, 20.9)
Pixel: (403, 282), MM: (41.0, 20.8)
Pixel: (403, 282), MM: (41.0, 20.8)
Pixel: (403, 282), MM: (41.0, 20.8)
Pixel: (403, 282), MM: (41.0, 20.8)
Pixel: (403, 282), MM: (41.0, 20.8)


KeyboardInterrupt: 

In [ ]:
import cv2
import math
import numpy as np
import time
from realsense_camera import *

# Инициализация камеры
camera = RealsenseCamera()

# Переменные для трекинга
cx, cy = 0, 0
cx_mm, cy_mm = 0, 0
prev_cx, prev_cy = 0, 0
stable_count = 0

# Параметры стабилизации
MIN_STABLE_FRAMES = 5
MAX_MOVEMENT = 15

# Параметры фильтрации контуров
MIN_CONTOUR_AREA = 500
MIN_CONTOUR_WIDTH = 20
MIN_CONTOUR_HEIGHT = 20

# Параметры камеры (должны быть получены из калибровки)
Cx = 320  # Оптический центр X
Cy = 240  # Оптический центр Y
fx = 604.602  # Фокусное расстояние X
fy = 604.162  # Фокусное расстояние Y

rotation=0
def getOrientation(pts, img):
    rect = cv2.minAreaRect(pts)
    width = int(rect[1][0])
    height = int(rect[1][1])
    angle = int(rect[2])
    if width < height:
        angle = 90- angle
    else:
        angle = 90+(90-angle)
    return angle,width, height

print("Запуск обработки видео. Нажмите 'q' для выхода.")

while True:
    # Получаем кадры
    ret, depth_frame, color_frame, depth = camera.get_frame_stream()

    if not ret:
        print("Ошибка получения кадра")
        continue

    height, width = color_frame.shape[0:2]   

    # Обработка цвета
    hsv = cv2.cvtColor(color_frame, cv2.COLOR_BGR2HSV)    

    # Цветовые диапазоны
    bin1 = cv2.inRange(hsv, (20, 100, 100), (30, 255, 255)) # желтый
    bin2 = cv2.inRange(hsv, (0, 59, 0), (21, 255, 255)) # красный
    bin3 = cv2.inRange(hsv, (35, 100, 100), (85, 255, 255)) # зеленый
    bin4 = cv2.inRange(hsv, (90, 100, 100), (130, 255, 255)) # синий
    binary = bin1 + bin2 + bin3 + bin4  

    # Морфологические операции
    kernel = np.ones((5, 5), np.uint8)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

    # Поиск контуров
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Фильтрация контуров
    valid_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < MIN_CONTOUR_AREA:
            continue

        x, y, w, h = cv2.boundingRect(contour)
        if w < MIN_CONTOUR_WIDTH or h < MIN_CONTOUR_HEIGHT:
            continue

        valid_contours.append(contour)

    # Обработка найденных контуров
    frame_display = color_frame.copy()

    if valid_contours:
        largest_contour = max(valid_contours, key=cv2.contourArea)
        moments = cv2.moments(largest_contour)

        if moments["m00"] > 0: 
            new_cx = int(moments["m10"] / moments["m00"])  
            new_cy = int(moments["m01"] / moments["m00"])

            # Проверка стабильности
            movement = math.sqrt((new_cx - prev_cx)**2 + (new_cy - prev_cy)**2)

            if movement < MAX_MOVEMENT:
                stable_count += 1
            else:
                stable_count = 0

            # Обновление координат при стабильности
            if stable_count >= MIN_STABLE_FRAMES:
                cx, cy = new_cx, new_cy

                # Преобразование в миллиметры с проверкой глубины
                if 0 <= cy < depth.shape[0] and 0 <= cx < depth.shape[1]:
                    depth_value = depth[cy, cx]
                    if depth_value > 0:
                        cx_mm = (cx - Cx) * depth_value / fx
                        cy_mm = (cy - Cy) * depth_value / fy
                    else:
                        cx_mm, cy_mm = 0, 0
                else:
                    cx_mm, cy_mm = 0, 0

            prev_cx, prev_cy = new_cx, new_cy
            
            rotation,width,height=getOrientation(largest_contour, frame_display)
            
            # Визуализация
            cv2.drawContours(frame_display, [largest_contour], -1, (0, 255, 0), 3)
            cv2.circle(frame_display, (cx, cy), 8, (0, 0, 255), -1)
            cv2.circle(frame_display, (cx, cy), 12, (255, 255, 255), 2)

            # Отображение информации
            cv2.putText(frame_display, f'X: {cx} px', (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            cv2.putText(frame_display, f'Y: {cy} px', (10, 60), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            cv2.putText(frame_display, f'X: {cx_mm:.1f} mm', (10, 90), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            cv2.putText(frame_display, f'Y: {cy_mm:.1f} mm', (10, 120), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            cv2.putText(frame_display, f'rotation: {rotation:.1f}', (10, 150), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)  

            print(f"Pixel: ({cx}, {cy}), MM: ({cx_mm:.1f}, {cy_mm:.1f})")

    cv2.imshow('Color Frame', frame_display)

    # Выход по 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
            
camera.release()
cv2.destroyAllWindows()

Запуск обработки видео. Нажмите 'q' для выхода.
Pixel: (0, 0), MM: (0.0, 0.0)
Pixel: (0, 0), MM: (0.0, 0.0)
Pixel: (0, 0), MM: (0.0, 0.0)
Pixel: (0, 0), MM: (0.0, 0.0)
Pixel: (0, 0), MM: (0.0, 0.0)
Pixel: (0, 0), MM: (0.0, 0.0)
Pixel: (215, 139), MM: (-72.4, -69.7)
Pixel: (215, 139), MM: (-72.8, -70.0)
Pixel: (215, 139), MM: (-72.9, -70.2)
Pixel: (215, 139), MM: (-72.8, -70.0)
Pixel: (215, 139), MM: (-74.3, -71.6)
Pixel: (215, 139), MM: (-72.1, -69.4)
Pixel: (215, 139), MM: (-72.2, -69.5)
Pixel: (215, 139), MM: (-72.2, -69.5)
Pixel: (215, 139), MM: (-72.4, -69.7)
Pixel: (215, 139), MM: (0.0, 0.0)
Pixel: (215, 139), MM: (-72.6, -69.9)
Pixel: (215, 139), MM: (0.0, 0.0)
Pixel: (215, 139), MM: (-72.2, -69.5)
Pixel: (215, 139), MM: (-73.3, -70.5)
Pixel: (215, 139), MM: (-72.8, -70.0)
Pixel: (215, 139), MM: (-72.1, -69.4)
Pixel: (215, 139), MM: (0.0, 0.0)
Pixel: (215, 139), MM: (-72.8, -70.0)
Pixel: (215, 139), MM: (-72.8, -70.0)
Pixel: (215, 139), MM: (-74.2, -71.4)
Pixel: (215, 139), MM: (

Pixel: (215, 139), MM: (-72.1, -69.4)
Pixel: (215, 140), MM: (-73.8, -70.3)
Pixel: (215, 139), MM: (-72.4, -69.7)
Pixel: (215, 140), MM: (-72.6, -69.2)
Pixel: (215, 140), MM: (-72.2, -68.9)
Pixel: (215, 139), MM: (-72.1, -69.4)
Pixel: (215, 139), MM: (-72.6, -69.9)
Pixel: (215, 139), MM: (-73.1, -70.4)
Pixel: (215, 140), MM: (-72.4, -69.0)
Pixel: (215, 139), MM: (-73.1, -70.4)
Pixel: (215, 140), MM: (-72.8, -69.4)
Pixel: (215, 139), MM: (-73.5, -70.7)
Pixel: (215, 139), MM: (0.0, 0.0)
Pixel: (215, 140), MM: (-72.8, -69.4)
Pixel: (215, 140), MM: (-72.2, -68.9)
Pixel: (215, 139), MM: (0.0, 0.0)
Pixel: (215, 140), MM: (-73.1, -69.7)
Pixel: (215, 139), MM: (-72.6, -69.9)
Pixel: (215, 139), MM: (-72.6, -69.9)
Pixel: (215, 140), MM: (-72.8, -69.4)
Pixel: (215, 140), MM: (-73.1, -69.7)
Pixel: (215, 139), MM: (-72.9, -70.2)
Pixel: (215, 140), MM: (-72.2, -68.9)
Pixel: (215, 139), MM: (-73.5, -70.7)
Pixel: (215, 140), MM: (-72.4, -69.0)
Pixel: (215, 139), MM: (-73.3, -70.5)
Pixel: (215, 139), M

Pixel: (215, 139), MM: (-72.6, -69.9)
Pixel: (215, 139), MM: (-72.4, -69.7)
Pixel: (215, 139), MM: (-73.1, -70.4)
Pixel: (215, 139), MM: (-72.2, -69.5)
Pixel: (215, 140), MM: (-72.9, -69.5)
Pixel: (215, 139), MM: (-73.6, -70.9)
Pixel: (215, 139), MM: (-73.1, -70.4)
Pixel: (215, 139), MM: (-72.9, -70.2)
Pixel: (215, 139), MM: (0.0, 0.0)
Pixel: (215, 139), MM: (-72.9, -70.2)
Pixel: (215, 140), MM: (-73.5, -70.0)


In [1]:
import cv2
import math
import numpy as np
import time
from realsense_camera import *
from kuka import Kuka
from openshowvar import *

#Connect robot
robot = openshowvar(ip = '192.168.17.2', port = 7000)
kuka = Kuka(robot)

kuka.set_base(8)
kuka.set_tool(10)

# Set speed (%)
kuka.set_speed(50)

kuka.read_cartesian()
x0=282.73
y0=122.48
z0=451.23
# Чтение текущих координат робота
kuka.read_cartesian()
# Построение траектории
first_point=[x0, y0, z0, 90.57, 0, 180]
trajectory=np.array([first_point])
kuka.lin_continuous(kuka,trajectory)

# Инициализация камеры
camera = RealsenseCamera()

# Переменные для трекинга
cx, cy = 0, 0
cx_mm, cy_mm = 0, 0
stable_count = 0

# Параметры стабилизации
MIN_STABLE_FRAMES = 5
MAX_MOVEMENT = 15

# Параметры фильтрации контуров
MIN_CONTOUR_AREA = 500
MIN_CONTOUR_WIDTH = 20
MIN_CONTOUR_HEIGHT = 20

# Параметры камеры (должны быть получены из калибровки)
Cx = 320  # Оптический центр X
Cy = 240  # Оптический центр Y
fx = 604.602  # Фокусное расстояние X
fy = 604.162  # Фокусное расстояние Y
rotation=0
CX_mm=0
CY_mm=0
def getOrientation(pts, img):
    rect = cv2.minAreaRect(pts)
    width = int(rect[1][0])
    height = int(rect[1][1])
    angle = int(rect[2])
    if width < height:
        angle = 90- angle
    else:
        angle = 90+(90-angle)
    return angle,width, height

print("Запуск обработки видео. Нажмите 'q' для выхода.")

while True:
    # Получаем кадры
    ret, depth_frame, color_frame, depth = camera.get_frame_stream()

    if not ret:
        print("Ошибка получения кадра")
        continue

    height, width = color_frame.shape[0:2]   

    # Обработка цвета
    hsv = cv2.cvtColor(color_frame, cv2.COLOR_BGR2HSV)    

    # Цветовые диапазоны
    bin1 = cv2.inRange(hsv, (20, 100, 100), (30, 255, 255)) # желтый
    bin2 = cv2.inRange(hsv, (0, 59, 0), (21, 255, 255)) # красный
    bin3 = cv2.inRange(hsv, (35, 100, 100), (85, 255, 255)) # зеленый
    bin4 = cv2.inRange(hsv, (90, 100, 100), (130, 255, 255)) # синий
    binary = bin1 + bin2 + bin3 + bin4  

    # Морфологические операции
    kernel = np.ones((5, 5), np.uint8)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

    # Поиск контуров
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Фильтрация контуров
    valid_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < MIN_CONTOUR_AREA:
            continue

        x, y, w, h = cv2.boundingRect(contour)
        if w < MIN_CONTOUR_WIDTH or h < MIN_CONTOUR_HEIGHT:
            continue

        valid_contours.append(contour)

    # Обработка найденных контуров
    frame_display = color_frame.copy()

    if valid_contours:
        largest_contour = max(valid_contours, key=cv2.contourArea)
        moments = cv2.moments(largest_contour)

        if moments["m00"] > 0: 
            cx = int(moments["m10"] / moments["m00"])  
            cy = int(moments["m01"] / moments["m00"])

            # Преобразование в миллиметры с проверкой глубины
            if 0 <= cy < depth.shape[0] and 0 <= cx < depth.shape[1]:
                depth_value = depth[cy, cx]
                if depth_value > 0:
                    cx_mm = (Сx - сx) * depth_value / fx
                    cy_mm = (Сy - сy) * depth_value / fy
                    rotation,width,height=getOrientation(largest_contour, frame_display)
                    CX_mm=cx_mm+30
                    CY_mm=cy_mm+65
                    depth_value=depth[cy, cx]-452
                    print(CX_mm, CY_mm, depth_value)
                    
                    # Движение Kuka
                    kuka.open_grip()
                    # Чтение текущих координат робота
                    kuka.read_cartesian()
                    # Построение траектории
                    first_point=[kuka.x_cartesianCX_mm, kuka.y_cartesian-CY_mm, depth_value,kuka.A_cartesian, 0, kuka.C_cartesian]
                    second_point=[kuka.x_cartesian+CX_mm, kuka.y_cartesian-CY_mm, depth_value,rotation, 0, kuka.C_cartesian]
                    trajectory=np.array([first_point, second_point])
                    kuka.lin_continuous(kuka,trajectory)
                    kuka.close_grip()
                    time.sleep(1)
                    # Чтение текущих координат робота
                    kuka.read_cartesian()
                    # Построение траектории
                    first_point=[x0, y0, z0, 90.57, 0, 180]
                    trajectory=np.array([first_point])
                    kuka.lin_continuous(kuka,trajectory)
                    frame_display=color_frame
                else:
                    cx_mm, cy_mm = 0, 0
            else:
                cx_mm, cy_mm = 0, 0
                
            # Визуализация
            cv2.drawContours(frame_display, [largest_contour], -1, (0, 255, 0), 3)
            cv2.circle(frame_display, (cx, cy), 8, (0, 0, 255), -1)
            cv2.circle(frame_display, (cx, cy), 12, (255, 255, 255), 2)

            # Отображение информации
            cv2.putText(frame_display, f'X: {cx} px', (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            cv2.putText(frame_display, f'Y: {cy} px', (10, 60), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            cv2.putText(frame_display, f'X: {cx_mm:.1f} mm', (10, 90), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            cv2.putText(frame_display, f'Y: {cy_mm:.1f} mm', (10, 120), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            cv2.putText(frame_display, f'rotation: {rotation:.1f}', (10, 150),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)  

            print(f"Pixel: ({cx}, {cy}), MM: ({CX_mm:.1f}, {CY_mm:.1f})")

    cv2.imshow('Color Frame', frame_display)

    # Выход по 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
            
camera.release()
cv2.destroyAllWindows()

Запуск обработки видео. Нажмите 'q' для выхода.
-30.0 -71.32611782932392 94
Pixel: (320, 233), MM: (-30.0, -71.3)
10.765660715644344 -30.865115647789835 51
Pixel: (369, 281), MM: (10.8, -30.9)
-205.7519822957913 -25.636385605185367 54


KeyboardInterrupt: 